In [ ]:
from dotenv import load_dotenv  
import os  
# Configure logger
import logging
import sys
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import CodeInterpreterTool
from azure.ai.agents.models import ListSortOrder
from azure.ai.agents.models import McpTool, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval

# Load the .env file  
load_dotenv(override=True) 

# Now you can use the environment variables, for example:  
project_endpoint = os.environ["PROJECT_ENDPOINT"]
model=os.environ["MODEL_DEPLOYMENT_NAME"]

print(project_endpoint + " " + model)

In [ ]:
# Retrieve the project endpoint from environment variables
project_endpoint = os.environ["PROJECT_ENDPOINT"]


# Initialize the AIProjectClient
project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
)

In [ ]:

agent_name = os.environ["AZURE_MCP_AGENT_NAME"]
agent_list = project_client.agents.list_agents()

agent = None
if agent_list:
    for agent_object in agent_list:
        if agent_object.name == agent_name:
            agent = agent_object

if not agent:
    print(f"Agent '{agent_name}' not found in the project.")
else:
    print(f"Agent '{agent_name}' found in the project.")
    print(f"Agent ID: {agent.id}")

In [ ]:
mcp_server_url = "https://gitmcp.io/Azure/azure-rest-api-specs"
mcp_server_label = "rest_api_specs"

project_client = AIProjectClient(
    endpoint=project_endpoint,
    credential=DefaultAzureCredential(),
    #exclude_environment_credential=True,
    #exclude_managed_identity_credential=True
)

agents_client = project_client.agents

In [ ]:
if not agent:
    print(f"Agent with name '{agent_name}' not found, creating a new agent.")

    # Initialize agent MCP tool
    mcp_tool = McpTool(
    server_label=mcp_server_label,
    server_url=mcp_server_url,
    allowed_tools=[],  # Optional: specify allowed tools
    )

    print(f"Allowed tools: {mcp_tool.allowed_tools}")

    # Create a new agent with the mcp tool definitions
    agent = agents_client.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name=agent_name,
        instructions="""
        You are a helpful assistant. Use the tools provided to answer the user's questions.  Be sure to cite your sources.""",
        tools=mcp_tool.definitions,
)
    print(f"Created agent, ID: {agent.id}")
else:
    print(f"Found agent by name '{agent_name}', ID={agent.id}")



In [ ]:
thread = agents_client.threads.create()
print(f"Created thread, ID: {thread.id}")

In [ ]:
message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
    TEll me about the Azure REST API specs
    """
)

print(f"Created message, ID: {message.id}")

In [ ]:
run = project_client.agents.runs.create(thread_id=thread.id, agent_id=agent.id, tool_resources=mcp_tool.resources)
print(f"Created run, ID: {run.id}")

In [ ]:
import time

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    print(f"Waiting for run to complete, current status: {run.status}")
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
        tool_calls = run.required_action.submit_tool_approval.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
            break

        tool_approvals = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredMcpToolCall):
                try:
                    print(f"Approving tool call: {tool_call}")
                    tool_approvals.append(
                        ToolApproval(
                            tool_call_id=tool_call.id,
                            approve=True,
                            headers=mcp_tool.headers,
                        )
                    )
                except Exception as e:
                    print(f"Error approving tool_call {tool_call.id}: {e}")

        print(f"tool_approvals: {tool_approvals}")
        if tool_approvals:
            agents_client.runs.submit_tool_outputs(
                thread_id=thread.id, run_id=run.id, tool_approvals=tool_approvals
            )

print(f"Current run status: {run.status}")


In [ ]:
if run.status == "failed":
        print(f"Run failed: {run.last_error}")
    
# Fetch and log all messages
messages = project_client.agents.messages.list(thread_id=thread.id)
for message in messages:
    print(f"Role: {message.role}, Content: {message.content}")